In [2]:
import pandas as pd
import io
import tensorflow as tf
# !pip install tensorflow_hub
import tensorflow_hub as hub
# !pip install tensorflow_text
import tensorflow_text as text


import matplotlib.pyplot as plt
import torch
# !pip install transformers
import transformers
from transformers import BertTokenizer, BertForSequenceClassification


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 KB 4.7 MB/s eta 0:00:00
You should consider upgrading via the '/home/sumedhshah94/anaconda3/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 35.6 MB/s eta 0:00:0000:0100:01


You should consider upgrading via the '/home/sumedhshah94/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/sumedhshah94/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
#!pip install tensorflow-gpu
!pip install --user --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.6 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/home/sumedhshah94/anaconda3/bin/python -m pip install --upgrade pip' command.


## Load Data

In [3]:
songs = pd.read_csv("drive/MyDrive/W266_Final_Project/songs_charts_new_2000s.csv")
songs = songs.sort_values('song_year')

In [4]:
batch_1 = songs[['lyrics','hit']]
batch_1.head()

,lyrics,hit
0,NA My servants began to forge what was to beco...,0
1,NA NA Fuck bein' on some chill shit We go 0 to...,1
2,You keep on takin' from me But where's my ten ...,0
3,When the fever's gone And the road's to dust I...,0
4,"Bullets fly, legs buckle, bodies pile to the s...",0


In [5]:
batch_1['hit'].value_counts()

0    6978
1    1608
Name: hit, dtype: int64

In [12]:
# Split into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(batch_1['lyrics'],batch_1['hit'])

## Load BERT tokenizer and encoder

In [6]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1")

## Create neural network

In [7]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string,name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid',name='output')(l)

model = tf.keras.Model(inputs=[text_input], outputs= [l])

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [9]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


## Fit and Evaluate Model

In [13]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
  5/202 [..............................] - ETA: 49:41 - loss: 0.4763 - accuracy: 0.8188

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
# SOURCE 1: https://medium.com/@aniketmohan/text-classification-using-bert-and-tensorflow-on-google-colab-b222762f6b48
# SOURCE 2: https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/classify_text_with_bert.ipynb#scrollTo=aksj743St9ga